## Preambule

In [502]:
# General packages
import json
from pathlib import Path

# Plotting
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr
from plotly.colors import n_colors
from plotly.subplots import make_subplots
from tqdm import tqdm

## Read

In [503]:
path_main = Path("K:/Code/EffortSharing/")
path_data = Path("K:/Data/Data_EffortSharing/DataUpdate_ongoing/startyear_2021/")
path_data_gen = Path("K:/Data/Data_EffortSharing/DataUpdate_ongoing/")
path_figs = path_main / "Figures" / "Presentations"
name_gas = "GHG"
name_lulucf = "incl"
name_histvar = name_gas + "_hist"
if name_lulucf == "excl":
    name_histvar = name_gas + "_hist" + name_lulucf

In [504]:
xr_dataread = xr.open_dataset(path_data / "xr_dataread.nc")
all_regions_iso = np.load(path_data_gen / "all_regions.npy")
all_regions_names = np.load(path_data_gen / "all_regions_names.npy")
all_countries_iso = np.load(path_data_gen / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data_gen / "all_countries_names.npy", allow_pickle=True)

In [505]:
df_nz = pd.read_csv("X:/user/dekkerm/Data/NetZeroYears/nztargetyears.csv")
xr_nz = xr.Dataset.from_dataframe(df_nz.set_index("iso3c")[["end_target_year_comb"]]).rename(
    {"iso3c": "Region", "end_target_year_comb": "NZ_year"}
)
regs = np.array(xr_nz.Region)
regs[regs == "FRA"] = "EU"
xr_nz = xr_nz.assign_coords({"Region": regs})
policy_budget = (xr_nz.NZ_year - 2021) * 0.5 * xr_dataread[name_histvar].sel(Time=2021)

In [506]:
major_countries = [
    "USA",
    "JPN",
    "EU",
    "KOR",
    "BRA",
    "ZAF",
    "CHN",
    "IDN",
    "MAR",
    "IND",
    "ZWE",
    "UGA",
][::-1]

In [507]:
settings_default = {
    "Temperature": [2.0],
    "Risk": [0.5],
    "NegEmis": 0.5,
    "NonCO2red": [0.1, 0.33, 0.5, 0.67, 0.9],
    "Timing": ["Delayed", "Immediate"],
    "Convergence_year": [2040, 2050, 2060],
    #'Discount_factor': [0.0, ],
    "Scenario": ["SSP1", "SSP2", "SSP3", "SSP4", "SSP5"],
    #'Historical_startyear': [1990]
}
figname = "Figure_majoremitters_nonco2_conv_scen_resp_tim_20_50.svg"

In [508]:
xr_allocs = []
for cty in major_countries:  # list(all_countries_iso)+['EU']+['EARTH']:
    xr_allocs.append(
        xr.open_dataset(
            path_data
            / ("Allocations_" + name_gas + "_" + name_lulucf)
            / ("xr_alloc_" + cty + ".nc")
        )
        .expand_dims({"Region": [cty]})
        .sel(**settings_default)[["GF", "ECPC", "PCC", "AP", "PC"]]
    )
xr_allocs = xr.merge(xr_allocs)

In [509]:
reds_2040 = (
    -(xr_allocs.sel(Time=2040) - xr_dataread.sel(Time=2015)[name_histvar])
    / xr_dataread.sel(Time=2015)[name_histvar]
)
reds_2030 = (
    -(xr_allocs.sel(Time=2030) - xr_dataread.sel(Time=2015)[name_histvar])
    / xr_dataread.sel(Time=2015)[name_histvar]
)
# xr_bud = xr_allocs.where(xr_allocs >=0, 0).sum(dim='Time')
# xr_bud3 = xr_allocs.where(xr_allocs.Time <= xr_nz.NZ_year).sum(dim='Time')
# debtpc = (xr_bud3 - policy_budget) / xr_dataread.Population.sel(Time=2021, Scenario='SSP2')
vals_2030_pc = []
vals_2030_ecpc = []
vals_2030_ap = []
vals_2040_pc = []
vals_2040_ecpc = []
vals_2040_ap = []
regs_pcc = []
regs_ecpc = []
regs_ap = []
debt_pc = []
debt_ecpc = []
debt_ap = []
for cty in major_countries:
    vals_2030_pc = vals_2030_pc + list(reds_2030.sel(Region=cty).PCC.values.flatten())
    vals_2030_ecpc = vals_2030_ecpc + list(reds_2030.sel(Region=cty).ECPC.values.flatten())
    vals_2030_ap = vals_2030_ap + list(reds_2030.sel(Region=cty).AP.values.flatten())

    vals_2040_pc = vals_2040_pc + list(reds_2040.sel(Region=cty).PCC.values.flatten())
    vals_2040_ecpc = vals_2040_ecpc + list(reds_2040.sel(Region=cty).ECPC.values.flatten())
    vals_2040_ap = vals_2040_ap + list(reds_2040.sel(Region=cty).AP.values.flatten())

    # debt_pc = debt_pc + list(debtpc.sel(Region=cty).PC.values.flatten())
    # debt_ecpc = debt_ecpc + list(debtpc.sel(Region=cty).ECPC.values.flatten())
    # debt_ap = debt_ap + list(debtpc.sel(Region=cty).AP.values.flatten())

    regs_pcc = regs_pcc + [all_regions_names[all_regions_iso == cty][0]] * len(
        reds_2030.sel(Region=cty).PCC.values.flatten()
    )
    regs_ecpc = regs_ecpc + [all_regions_names[all_regions_iso == cty][0]] * len(
        reds_2030.sel(Region=cty).ECPC.values.flatten()
    )
    regs_ap = regs_ap + [all_regions_names[all_regions_iso == cty][0]] * len(
        reds_2030.sel(Region=cty).AP.values.flatten()
    )

## Plot

In [510]:
fig = make_subplots(
    rows=1,
    cols=2,
    specs=[[{}, {}]],
    subplot_titles=(
        "<b>GHG reduction targets<br>by 2030 (w.r.t. 2015)",
        "<b>GHG reduction targets<br>by 2040 (w.r.t. 2015)",
    ),
    horizontal_spacing=0.05,
    vertical_spacing=0.05,
    shared_xaxes=False,
    shared_yaxes=True,
)

col1 = "#afc50f"  # "rgba(255, 65, 54, 1)"
col2 = "#2e5061"  #'rgba(61, 153, 112, 1)'
col3 = "#0062a6"
lw = 5

fig.add_trace(
    go.Box(
        x=vals_2030_pc,
        y=regs_pcc,
        name="Equality",
        boxpoints=False,
        offsetgroup="A",
        marker_color=col3,
        line_width=lw,
    ),
    1,
    1,
)
fig.add_trace(
    go.Box(
        x=vals_2030_ap,
        y=regs_ap,
        name="Capability",
        boxpoints=False,
        offsetgroup="B",
        marker_color=col1,
        line_width=lw,
    ),
    1,
    1,
)
fig.add_trace(
    go.Box(
        x=vals_2030_ecpc,
        y=regs_ecpc,
        name="Responsibility",
        boxpoints=False,
        offsetgroup="C",
        marker_color=col2,
        line_width=lw,
    ),
    1,
    1,
)  # rgba(255, 133, 27,1)

fig.add_trace(
    go.Box(
        x=vals_2040_pc,
        y=regs_pcc,
        offsetgroup="A",
        showlegend=False,
        boxpoints=False,
        marker_color=col3,
        line_width=lw,
    ),
    1,
    2,
)
fig.add_trace(
    go.Box(
        x=vals_2040_ap,
        y=regs_ap,
        offsetgroup="B",
        showlegend=False,
        boxpoints=False,
        marker_color=col1,
        line_width=lw,
    ),
    1,
    2,
)
fig.add_trace(
    go.Box(
        x=vals_2040_ecpc,
        y=regs_ecpc,
        offsetgroup="C",
        showlegend=False,
        boxpoints=False,
        marker_color=col2,
        line_width=lw,
    ),
    1,
    2,
)

fig.add_vline(x=-0.5, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=-0.25, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=0.25, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=0.5, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=0.75, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=1.25, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=1, line_width=4, line_dash="dash", line_color="black")

# fig.add_trace(go.Box(x=debt_pcc, y=regs_pcc, offsetgroup='A', showlegend=False, boxpoints=False, marker_color='rgba(61, 153, 112, 1)', line_width=1), 1, 3)
# fig.add_trace(go.Box(x=debt_ap, y=regs_ap, offsetgroup='B', showlegend=False, boxpoints=False, marker_color=col1, line_width=3), 1, 3)
# fig.add_trace(go.Box(x=debt_ecpc, y=regs_ecpc, offsetgroup='C', showlegend=False, boxpoints=False,  marker_color=col2, line_width=3), 1, 3)
fig.add_vline(x=0, line_width=4, line_dash="solid", line_color="black")

fig.update_layout(
    boxmode="group",  # group together boxes of the different traces for each value of x
    boxgap=0.25,  # Space between different categories (days)
    boxgroupgap=0.0,
    yaxis=dict(tickfont=dict(size=18)),  # Increase tick font size for first subplot
    yaxis2=dict(tickfont=dict(size=18), showline=False, ticks=""),
    # yaxis3=dict(tickfont=dict(size=18), showline=False, ticks=''),
    xaxis=dict(tickfont=dict(size=18)),  # Increase tick font size for first subplot
    xaxis2=dict(tickfont=dict(size=18)),
    # xaxis3=dict(tickfont=dict(size=18))
)

for cty in major_countries:
    name = all_regions_names[all_regions_iso == cty][0]
    vals_2030 = (
        list(np.array(vals_2030_pc)[np.where(np.array(regs_pcc) == name)[0]])
        + list(np.array(vals_2030_ap)[np.where(np.array(regs_ap) == name)[0]])
        + list(np.array(vals_2030_ecpc)[np.where(np.array(regs_ecpc) == name)[0]])
    )
    vals_2040 = (
        list(np.array(vals_2040_pc)[np.where(np.array(regs_pcc) == name)[0]])
        + list(np.array(vals_2040_ap)[np.where(np.array(regs_ap) == name)[0]])
        + list(np.array(vals_2040_ecpc)[np.where(np.array(regs_ecpc) == name)[0]])
    )
    fig.add_annotation(
        x=1.2,
        y=name,
        text=str(int(np.round(min(vals_2030) * 100, 0)))
        + "-"
        + str(int(np.round(max(vals_2030) * 100, 0)))
        + "%",
        showarrow=False,
        yshift=0,
        xshift=50,
    )
    fig.add_annotation(
        x=1.2,
        y=name,
        text=str(int(np.round(min(vals_2040) * 100, 0)))
        + "-"
        + str(int(np.round(max(vals_2040) * 100, 0)))
        + "%",
        showarrow=False,
        yshift=0,
        xshift=50,
        xref="x2",
    )

fig.update_layout(xaxis={"anchor": "y", "overlaying": "x", "side": "top"}, yaxis_domain=[0, 0.93])

fig.update_layout(xaxis2={"anchor": "y", "overlaying": "x", "side": "top"}, yaxis2_domain=[0, 0.93])

for i in range(2):
    fig.update_xaxes(
        tickvals=[
            [-0.5, -0.25, 0, 0.25, 0.5, 0.75, 1.0, 1.25],
            [-0.5, -0.25, 0, 0.25, 0.5, 0.75, 1.0, 1.25],
            [-300, -150, 0, 150],
        ][i],  # Custom tick positions
        ticktext=[
            ["-50%", "-25%", "0%", "25%", "50%", "75%", "Net<br>zero", "125%"],
            ["-50%", "-25%", "0%", "25%", "50%", "75%", "Net<br>zero", "125%"],
            [
                "300<br>t CO<sub>2</sub>/cap<br>debt",
                "150<br>t CO<sub>2</sub>/cap<br>debt",
                "No debt<br>nor credit",
                "150<br>t CO<sub>2</sub>/cap<br>credit",
            ],
        ][i],  # Custom tick labels
        row=1,
        col=1 + i,
        range=[[-0.5, 1.4], [-0.5, 1.4], [-300, 150]][i],
        # title_text = ["Reduction relative to 2015", "Reduction relative to 2015", "Carbon debt (left) or credit (right) per capita"][i],
    )

fig.update_traces(orientation="h")
for annotation in fig["layout"]["annotations"]:
    annotation["font"] = dict(size=25)
fig.update_layout(
    height=1100,
    width=2000,
    template="simple_white",
    legend=dict(x=-0.15, y=1.08, font=dict(size=20)),
)
fig.write_image(path_figs / figname, scale=4)
fig.show()

In [511]:
xr_allocs.close()